In [26]:
  
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re


# 크롤링 데이터 저장을 위함
title_text=[]
date_text=[]
result={}
    

def crawler(maxpage,query,sort,s_date,e_date):

    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    page=1
    
    # 400 페이지까지만 제공함.
    while page<=4001:
        # 네이버 기사형태 지정
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
    
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
 
        # 제목 추출
        atags = soup.select('ul.list_news > li.bx > div.news_wrap.api_ani_send > div > a')
        for atag in atags:
            title_text.append(atag.text)     #제목
              
        
         #날짜 추출 
        date_lists = soup.select('ul.list_news > li.bx > div.news_wrap.api_ani_send > div > div > div > span')
        for date_list in date_lists:
            # 가끔 이상한 info가 껴있는 경우가 있음.
            if len(date_list)!=1:
                continue
            date_text.append(date_list.text)
            
        
        
        #모든 리스트 딕셔너리형태로 저장
        # id= date , document = title , label 은 일단 3으로 고정(의미없는 값)
        result= {"id" : date_text , "document":title_text , "label":'3' }  

        
        df = pd.DataFrame(result)  #df로 변환
        
        page += 10
        
    
        print(df)
        
        
        # 인덱스 없애기
        df.to_csv(f'{query}_title.txt', sep = '\t', index = False)
        
        if (df['id'][len(df)-1] == '2021.06.17.'):
            return
        
        # 400페이지만 제공하므로 시작 날짜를 마지막에 크롤링한 것으로 바꾸고 다시 크롤링
        if(page == 4001 and df['id'][len(df)-1] != '2021.06.17.'):
            s_date = (df['id'][len(df)-1])
            crawler(maxpage,query,sort,s_date,e_date)
            
             
         
        
        
        
        
# url 인자들 설정
maxpage = "100000" 
query = ["에이비엘바이오","삼천당제약","안트로젠",
"압타바이오","유바이오로직스","보령제약"]

sort = "2"  #관련도순=0  최신순=1  오래된순=2
s_date = "2017.06.01"  # 시작 날짜 입력
e_date = "2021.06.17" # 마지막 날짜 입력

for i in query:
    title_text=[]
    date_text=[]
    result={}
    crawler(maxpage,i,sort,s_date,e_date)



            id                                  document label
0  2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1  2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2  2017.06.15.  [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3  2017.07.21.                        파킨슨병은 장(腸}에서 시작했다?     3
4  2017.07.26.                  동아에스티, 개방형 협력으로 혁신 신약 개발     3
5  2017.08.30.              에이비엘바이오, '이중항체' 항암신약 국내임상 돌입     3
6  2017.09.01.             에이비엘바이오, 이중항체 고형암 신약 임상 1상 돌입     3
7  2017.09.13.           바이오 메카 '인천', '2017 바이오플러스인천' 개최     3
8  2017.09.13.      한국바이오협회ㆍ인천시, '2017 바이오플러스인천' 컨퍼런스 진행     3
9  2017.09.14.           "바이오 신사업 규제 최소화…도전적인 R&D 장려하겠다"     3
             id                                    document label
0   2017.06.09.                            "바이오 투자 더 늘릴 계획"     3
1   2017.06.15.     [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2   2017.06.15.    [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3   2017.07.21.                          파킨

             id                                          document label
0   2017.06.09.                                  "바이오 투자 더 늘릴 계획"     3
1   2017.06.15.           [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2   2017.06.15.          [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3   2017.07.21.                                파킨슨병은 장(腸}에서 시작했다?     3
4   2017.07.26.                          동아에스티, 개방형 협력으로 혁신 신약 개발     3
5   2017.08.30.                      에이비엘바이오, '이중항체' 항암신약 국내임상 돌입     3
6   2017.09.01.                     에이비엘바이오, 이중항체 고형암 신약 임상 1상 돌입     3
7   2017.09.13.                   바이오 메카 '인천', '2017 바이오플러스인천' 개최     3
8   2017.09.13.              한국바이오협회ㆍ인천시, '2017 바이오플러스인천' 컨퍼런스 진행     3
9   2017.09.14.                   "바이오 신사업 규제 최소화…도전적인 R&D 장려하겠다"     3
10  2017.10.11.               제약·바이오 업계, 각광받는 차세대 기술 '이중항체' 개발 집중     3
11  2017.10.26.             [도약하는 K바이오] 블록버스터 신약 무대 "주인공은 나야 나 ~"     3
12  2017.11.06.                         동아에스티, 병원·학계 공동연구…연구역량 강

              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2    2017.06.15.  [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3    2017.07.21.                        파킨슨병은 장(腸}에서 시작했다?     3
4    2017.07.26.                  동아에스티, 개방형 협력으로 혁신 신약 개발     3
..           ...                                       ...   ...
115  2018.05.09.           동아에스티, 1분기 영업익 105억…전년비 113.4%↑     3
116  2018.05.09.   동아에스티, 1분기 영업이익 105억원… 전년 동기比 113.4% 증가     3
117  2018.05.09.               동아에스티, 매출 정체지만 영업익은 113% 증가     3
118  2018.05.09.               동아에스티 반등 지속...1분기 매출 1356억원     3
119  2018.05.09.          동아에스티, 영업이익 113.4% 증가…당기순이익 흑자전환     3

[120 rows x 3 columns]
              id                                   document label
0    2017.06.09.                           "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.    [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     

              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2    2017.06.15.  [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3    2017.07.21.                        파킨슨병은 장(腸}에서 시작했다?     3
4    2017.07.26.                  동아에스티, 개방형 협력으로 혁신 신약 개발     3
..           ...                                       ...   ...
215  2018.09.10.            항체신약 개발 에이비엘바이오, 코스닥 '예비심사' 청구     3
216  2018.09.10.                  노랑풍선·에이비엘바이오 코스닥 상장예심 청구     3
217  2018.09.10.         거래소, 에이비엘바이오 등 2개사 코스닥 상장 예비심사 청구     3
218  2018.09.10.           한국거래소, 노랑풍선·에이비엘바이오 코스닥 상장예심 청구     3
219  2018.09.10.           거래소, 노랑풍선·에이비엘바이오 등 코스닥 상장예심 신청     3

[220 rows x 3 columns]
              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2

              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2    2017.06.15.  [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3    2017.07.21.                        파킨슨병은 장(腸}에서 시작했다?     3
4    2017.07.26.                  동아에스티, 개방형 협력으로 혁신 신약 개발     3
..           ...                                       ...   ...
325  2018.11.09.      [장외주식] 공모완료 노바렉스의 상승세와 공모예정 싸이토젠의 반등     3
326  2018.11.12.    [장외주식] 공모예정 싸이토젠과 청구예정 바디프랜드의 눈에 띄는 상승     3
327  2018.11.13.      [GBF2018]"퍼스트무버 전략을 찾아라"…글로벌바이오포럼 개막     3
328  2018.11.13.                     에이비엘바이오 사업소개하는 이재천 상무     3
329  2018.11.13.                       이재천 에이비엘바이오 상무 사업소개     3

[330 rows x 3 columns]
              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2

...

              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2    2017.06.15.  [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3    2017.07.21.                        파킨슨병은 장(腸}에서 시작했다?     3
4    2017.07.26.                  동아에스티, 개방형 협력으로 혁신 신약 개발     3
..           ...                                       ...   ...
535  2018.12.18.       [SENTV] 에이비엘바이오 “TRIGR, 주주·사업모델 확실”     3
536  2018.12.18.                        오늘의 증시 메모[12월 19일]     3
537  2018.12.18.    상장 앞둔 에이비엘바이오, 기술이전 파트너사 악성 풍문에 `노심초사`     3
538  2018.12.18.         에이비엘바이오 “TRIGR 기술이전 관련 풍문, 사실 아냐”     3
539  2018.12.18.                       오늘의 증시 일정 (12월 19일)     3

[540 rows x 3 columns]
              id                                         document label
0    2017.06.09.                                 "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.          [BioS] 바이오CEO, 새 정부에 바라는 1

              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.   [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2    2017.06.15.  [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3    2017.07.21.                        파킨슨병은 장(腸}에서 시작했다?     3
4    2017.07.26.                  동아에스티, 개방형 협력으로 혁신 신약 개발     3
..           ...                                       ...   ...
635  2018.12.20.           바이오주 대부분 내려, 에이비엘바이오 파멥신 주가는 급등     3
636  2018.12.20.                  20일 코스닥 외국인 순매수 상위종목(확정)     3
637  2018.12.20.                   20일 코스닥 기관 순매수 상위종목(확정)     3
638  2018.12.20.                   20일 코스닥 개인 순매수 상위종목(확정)     3
639  2018.12.21.          삼바·셀트·한미… K바이오 대표들, JP모건 행사에 몰린다     3

[640 rows x 3 columns]
              id                                          document label
0    2017.06.09.                                  "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.           [BioS] 바이오CEO, 새 정부에 바라

              id                                           document label
0    2017.06.09.                                   "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.            [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2    2017.06.15.           [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3    2017.07.21.                                 파킨슨병은 장(腸}에서 시작했다?     3
4    2017.07.26.                           동아에스티, 개방형 협력으로 혁신 신약 개발     3
..           ...                                                ...   ...
735  2018.12.27.                              사상 최대치 갈아치운 특례상장의 명과암     3
736  2018.12.27.                       [마감]코스닥, 외인·기관 매도에도 3거래일만 반등     3
737  2018.12.27.                      바이오株 '1월 효과' 기대감…외인·기관 연말 러브콜     3
738  2018.12.27.  [ET투자뉴스][에이비엘바이오 지분 변동] 한국투자글로벌 외 2명 -1.53%p 감...     3
739  2018.12.27.                            27일 코스닥 순매매 기관 상위종목(잠정)     3

[740 rows x 3 columns]
              id                                           document label
0    2017.06.0

              id                                      document label
0    2017.06.09.                              "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.       [BioS] 바이오CEO, 새 정부에 바라는 1st "연대보증제 폐지"     3
2    2017.06.15.      [BioS] 일자리 부족하다는데..바이오CEO 95% "일할 사람 부족"     3
3    2017.07.21.                            파킨슨병은 장(腸}에서 시작했다?     3
4    2017.07.26.                      동아에스티, 개방형 협력으로 혁신 신약 개발     3
..           ...                                           ...   ...
835  2019.01.02.             개인투자자, 코스닥 매수상위 10종목 중 9개 마이너스 수익     3
836  2019.01.02.                  움츠렸던 제약바이오, JP모건 콘퍼런스로 ‘기지개’     3
837  2019.01.02.            -37%↓ vs 51%↑…코스닥 개미 `쪽박`, 기관 `대박`     3
838  2019.01.02.  [제약톡톡] 기술수출의 ‘불씨’ 지핀다…제약사들 ‘JP모건 헬스케어 컨퍼런...     3
839  2019.01.02.                 [IPO 주관] 미래에셋대우, 롯데정보통신 잡고 1위     3

[840 rows x 3 columns]
              id                                  document label
0    2017.06.09.                          "바이오 투자 더 늘릴 계획"     3
1    2017.06.15.  


KeyboardInterrupt

